# Automatic Differentiation

In this assignment, we will be implementing a class that supports [Automatic Differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) on scalar values.

Automatic Differentiation fundamentally relies on **chain rule** and **partial derivatives of composite functions** $f(x) = a(b(c(x))) \implies f' = a' b' c'$ for accumulating gradients.

As we saw in the lecture there are two approaches to implementing Automatic Differentation. **forward-mode/right-to-left** and **reverse-mode/left-to-right**.

### Forward-mode

We calculate the partial derivatives of functions in the order the functions are composed. For a function $f(x) = a(b(c(x)))$ this involves
1. Evaluate $c(x)$ and calculate $c' = \frac{\delta c}{\delta x}$.
2. Evaluate $b(c(x))$. Calculate $b' = \frac{\delta b}{\delta c}$ and use it to calculate $\frac{\delta b}{\delta x} = b' c' = \frac{\delta b}{\delta c} \frac{\delta c}{\delta x}$. $c(x)$ and $c'$ are available from step 1.
3. Similarly evaluate $a(b(c(x)))$ and calculate $\frac{\delta b}{\delta x}$. Use $b(c(x))$ and $b'c'$ from the previous step.
4. We now have $\frac{\delta f}{\delta x}$.

### Implementation

We will do a minimal implementation that only supports a single scalar variable, you can refer to [micrograd](https://github.com/karpathy/micrograd/tree/master) to get an idea of backward-mode implementation with multi-variable support by storing the computation DAG.

Complete the Python class "ValueFwd" to support forward-mode Automatic Differentiation as explained above.
1. The class should store two values, the value of the variable/constant and its gradient w.r.t the independent variable. (hint: the gradient should be initialized to 0 for constants and 1 for variables, we will only have a single variable).
2. **ValueFwd** should support basic arithmetic operations, similar to **torch.Tensor**. Complete the class methods to add this functionality to ValueFwd. "\_\_add\_\_" is already implemented to give you an idea.
3. **TODO** Complete the implementation of ValueFwd and run the test code. play around with the input to test function, you are also free to implement mode test cases.

In [84]:
import numpy as np
from math import sin, cos
import math
from numpy import log
from typing import List, Tuple
from numpy.testing import assert_almost_equal

# forward mode

In [52]:
class ValueFwd:
    """ stores a single scalar value and its gradient """
    def __init__(self, data, diff = 0):
        self.v = data
        self.d = diff

    def __add__(l, r):
        """ support for + on ValueFwd datatype.

         l + r returns f, such that f.v = l.v + r.v and f.d = f' = df/dx.
        
        Args:
            l (ValueFwd): ValueFwd to the left of +.
            r (ValueFwd): ValueFwd to the right of +.

        Returns:
            ValueFwd with updated value and gradient.
        """
        return ValueFwd(l.v + r.v, l.d + r.d)

    def __sub__(l, r):
        """ support for - on ValueFwd datatype.

         l - r returns f, such that f.v = l.v - r.v and f.d = f' = df/dx.
        
        Args:
            l (ValueFwd): ValueFwd to the left of -.
            r (ValueFwd): ValueFwd to the right of -.

        Returns:
            ValueFwd with updated value and gradient.
        """
        pass

    def __mul__(l, r):
        """ support for * on ValueFwd datatype.

         l * r returns f, such that f.v = l.v * r.v and f.d = f' = df/dx.
        
        Args:
            l (ValueFwd): ValueFwd to the left of *.
            r (ValueFwd): ValueFwd to the right of *.

        Returns:
            ValueFwd with updated value and gradient.
        """
        pass

    def __truediv__(l, r):
        """ support for / on ValueFwd datatype.

         l / r returns f, such that f.v = l.v / r.v and f.d = f' = df/dx.
        
        Args:
            l (ValueFwd): ValueFwd to the left of /.
            r (ValueFwd): ValueFwd to the right of /.

        Returns:
            ValueFwd with updated value and gradient.
        """
        pass

    def sin(self):
        """ support for self.sin() on ValueFwd datatype.

        Args:
            self (ValueFwd): ValueFwd.

        Returns:
            returns ValueFwd f, such that f.v = sin(self.v) and f.d = f' = df/dx
        """
        pass

    def cos(self):
        """ support for self.cos() on ValueFwd datatype.

        Args:
            self (ValueFwd): ValueFwd.

        Returns:
            returns ValueFwd f, such that f.v = cos(self.v) and f.d = f' = df/dx
        """
        pass

    def exp(self, b):
        """ support for b^self on ValueFwd datatype.

            self.exp(math.e) = e^self
            
        Args:
            self (ValueFwd): ValueFwd.
            b (Number): A numerical value.

        Returns:
            returns ValueFwd f, such that f.v = b^(self.v) and f.d = f' = df/dx
        """
        pass
    
    def pow(self, p):
        """ support for pow on ValueFwd datatype.
            
            self.pow(3) = self^3
        
        Args:
            self (ValueFwd): ValueFwd.
            p (Number): A numerical value.

        Returns:
            returns ValueFwd f, such that f.v = (self.v)^p and f.d = f' = df/dx
        """
        pass

    def __repr__(self):
        return f"v: {self.v}, d:{self.d}"

# Test forward-mode automatic differentiation

Test function 1 is: `test_f1(x)`
$$f(x) = \sin\left( \frac{\sqrt{e^x + 2}}{2}\right)$$

The gradient for Test 1 is: `test_df1`
$$\frac{e^x \cos\left( \frac{\sqrt{e^x + 2}}{2} \right)}{4 \sqrt{e^x + 2}}$$


In [92]:
def test_f1(x):
    return ((x.exp(math.e) + ValueFwd(2)).pow(0.5)/ValueFwd(2)).sin()

def test_df1(x):
    ex = math.exp(x)
    num = ex*cos(math.sqrt(ex+2)/2)
    den = 4 * math.sqrt(ex+2)
    return num/den

## test Froward-mode Automatic Differentiation

In [ ]:
assert_almost_equal(test_f1(ValueFwd(1, 1)).d, test_df1(1))

# reverse mode

### Reverse-mode

We do two passes in the reverse mode of Automatic Differentiation. One forward and one backward.

For a function $f(x) = a(b(c(x)))$. We first calculate the local partial derivatives e.g. $\frac{\delta c}{\delta x}, \frac{\delta b}{\delta c}, \frac{\delta a}{\delta b}$ during the forward pass. and accumulate them in the reverse pass to get $\frac{\delta f}{\delta x}$
1. Evaluate $c(x)$ and calculate $c' = \frac{\delta c}{\delta x}$. Store the local gradient $c'$ for future use during the backward pass.
2. Evaluate $b(c(x))$. Calculate $b' = \frac{\delta b}{\delta c}$ and store it for future use during the backward pass.
3. Similarly evaluate $a(b(c(x)))$ and calculate $\frac{\delta a}{\delta b}$ and store it.
4. Initiate the backward pass to accumulate the local gradients and calculate $\frac{\delta f}{\delta x}$ 

### Implementation

Reverse mode is a little more complicated than the forward mode. Here gradients are accumulated during the backward pass. so, we also have to store the computation DAG(Directed Acyclic Graph) so that the backward pass can happen in the correct order.

Complete the Python class "ValueBwd" to support Reverse-mode Automatic Differentiation as explained above.
1. Similar to forward mode store the value of the variable/constant and its gradient w.r.t the independent variable(not updated in the forward pass). Since gradients are accumulated in the backward pass from output-to-input / left-to-right, we have to store the local gradients as well as dependencies i.e., what nodes does the current node depend on? Use `self._prev` list to store the **(input node, and local gradient)** tuples for the current node.
2. During the backward pass `backward` recursively propagate the gradients backward and populate `self.d` for the nodes.

In [67]:
class ValueBwd:
    """ stores a single scalar value and its gradient, updates gradient through back-prop """
    def __init__(self, data, prev = []):
        """ Value datatype that supports backpropagation on a single scalar variable.

        Args:
            data (Number): value of the variable/constant.
            prev (List[Tuple[ValueBwd, Number]]): List of node(ValueBwd), local gradient tuples that the current node depends on. 
                Empty list if the computational graph contains a single node.
        Returns:
            ValueBwd.
        """
        self.v = data
        self.d = 0
        self._prev = prev

    def __add__(l, r):
        """ support for + on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of +.
            r (ValueBwd): ValueBwd to the right of +.

        Returns:
            ValueBwd with updated value and gradient.
        """
        return ValueBwd(l.v + r.v, [(l, 1), (r, 1)])

    def __sub__(l, r):
        """ support for - on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of -.
            r (ValueBwd): ValueBwd to the right of -.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        pass

    def __mul__(l, r):
        """ support for * on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of -.
            r (ValueBwd): ValueBwd to the right of -.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        pass

    def __truediv__(l, r):
        """ support for / on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of -.
            r (ValueBwd): ValueBwd to the right of -.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        pass

    def sin(self):
        """ support for sin(self) on ValueBwd datatype.

        Args:

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        pass

    def cos(self):
        """ support for cos(self) on ValueBwd datatype.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        pass

    def exp(self, b):
        """ support for b^self on ValueBwd datatype.

        Args:
            b (Number):  A numerical value.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        pass
    
    def pow(self, p):
        """ support for self^p on ValueBwd datatype.

        Args:
            p (ValueBwd):  A numerical value.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        pass

    def backward(self, d):
        """ Updates local gradient and iteratively calls backward on self._prev.

        Args:
            d (Number): Gradient Value.

        Returns:
            None
        """
        pass

    def __repr__(self):
        return f"v: {self.v}, d:{self.d}"

## Test reverse mode AD

In [81]:
def test_f1(x):
    return ((x.exp(math.e) + ValueBwd(2)).pow(0.5)/ValueBwd(2)).sin()

def test_df1(x):
    ex = math.exp(x)
    num = ex*cos(math.sqrt(ex+2)/2)
    den = 4 * math.sqrt(ex+2)
    return num/den

In [90]:
x = 2
inp = ValueBwd(x)
o = test_f1(inp)
o.backward(1)

In [91]:
assert_almost_equal(inp.d, test_df1(2))